In [1]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Interaction
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
import plotly.express as px
import datetime

In [2]:
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '8g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/10 05:57:41 WARN Utils: Your hostname, DESKTOP-3NQ3PQI resolves to a loopback address: 127.0.1.1; using 172.17.18.177 instead (on interface eth0)
22/10/10 05:57:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 05:57:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
weights_sdf = spark.read.parquet(
    '../data/curated/demographic_weights.parquet'
)
consumers_sdf = spark.read.parquet(
    '../data/curated/cleaned_consumers.parquet'
)

user_details_sdf = spark.read.parquet(
    '../data/tables/consumer_user_details.parquet'
)

merchants_sdf = spark.read.parquet(
    '../data/curated/merchants.parquet'
)

postcode_poa_sdf = spark.read.parquet(
    '../data/curated/census/postcode_poa.parquet'
)


transactions_sdf = spark.read.parquet(
    '../data/tables/transactions_20210228_20210827_snapshot'
).union(
    spark.read.parquet(
        '../data/tables/transactions_20210828_20220227_snapshot'
    )
).union(
    spark.read.parquet(
        '../data/tables/transactions_20220228_20220828_snapshot'
    )
)

In [4]:
mean_weights_sdf = transactions_sdf.join(
    user_details_sdf,
    on = 'user_id',
    how = 'left'
).join(
    consumers_sdf.select(
        'consumer_id', 'postcode', 'gender'
    ),
    on = 'consumer_id',
    how = 'left'
).join(
    postcode_poa_sdf,
    on = 'postcode',
    how = 'left'
).join(
    weights_sdf,
    on = ['poa' ,'gender'],
    how = 'left'
).groupby(
    'merchant_abn'
).agg(
    {'weight' : 'mean'}
)

In [5]:
predictions_df = pd.read_parquet('../data/curated/lr_transaction_predictions.parquet')
mean_weights_df = mean_weights_sdf.toPandas()

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/root/.virtualenvs/ads_proj2/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/root/.virtualenvs/ads_proj2/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [ ]:
merchant_totals_df = pd.merge(
    predictions_df.groupby('merchant_abn')['weighted_dollar_value'].sum(),
    mean_weights_df,
    on = 'merchant_abn',

)

In [ ]:
total_predictions_df = predictions_df.groupby(
    'merchant_abn'
)['weighted_dollar_value'].sum().reset_index()

In [ ]:
px.histogram(total_predictions_df, 'weighted_dollar_value')

In [ ]:
px.scatter(pd.merge(total_predictions_df, merchants_df, on = 'merchant_abn'), x = 'take_rate', y = 'weighted_dollar_value', trendline = 'ols', hover_data = ['merchant_abn'])

In [ ]:
total_predictions_df['band'] = total_predictions_df['weighted_dollar_value'] < 20000

In [ ]:
predictions_df = pd.merge(
    predictions_df,
    total_predictions_df[['merchant_abn', 'band']],
    on = 'merchant_abn'
)

In [ ]:
px.scatter(predictions_df.groupby(['order_datetime', 'band'])['weighted_dollar_value'].mean().reset_index(), x = 'order_datetime', y = 'weighted_dollar_value', color = 'band')

In [ ]:
predictions_df